In [1]:
#!/home/akhanal1/Spring2018/pl-env/bin/python3.5
import os
os.chdir('/home/ak/Spring2018/ature')
import sys
import numpy as np
import itertools as itr
from random import shuffle
from commons.segmentation import AtureTest
from PIL import Image as IMG
from commons.IMAGE import Image, HighResolutionFundusImage
import cv2
import utils.filter_utils as filutils
%load_ext autoreload
%autoreload 2
sep = os.sep

In [2]:
########## PATHS ###########
Dirs = {}

Dirs['data']      = 'data'+sep+'hrf'

Dirs['images']    = Dirs['data'] +sep+ 'images'
Dirs['mask']      = Dirs['data'] +sep+ 'mask'
Dirs['truth']     = Dirs['data'] +sep+ 'manual1'
Dirs['segmented'] = Dirs['data'] +sep+ 'drive_segmented'
Dirs['patches']   = Dirs['data'] +sep+ 'patches'

for k, folder in Dirs.items():
    os.makedirs(folder, exist_ok=True)

def get_mask_file(file_name): 
    return file_name.split('.')[0] + '_mask' + '.tif' 

def get_ground_truth_file(file_name): 
    return file_name.split('.')[0] + '.tif'

input_image = '01_dr.JPG'
########## Paths end

########### PARAMS ##########
BILATERAL_KERNEL_SIZE = 61
BILATERAL_SIGMA_COLOR = 20
BILATERAL_SIGMA_SPACE = 20

kernels1 = filutils.build_filter_bank(k_size=75, gamma=0.7, lambd=10, sigma=4, orientations=64, psi=0)
kernels2 = filutils.build_filter_bank(k_size=75, gamma=0.7, lambd=20, sigma=8, orientations=64, psi=0)
kernels3 = filutils.build_filter_bank(k_size=75, gamma=0.7, lambd=38, sigma=14, orientations=64, psi=0)
kernels = kernels1 + kernels2 + kernels3

params = {'sk_threshold': 80,
          'alpha': 6.0,
          'orig_contrib': 0.8,
          'seg_threshold': 12}
##### Params end

In [ ]:
img_obj = HighResolutionFundusImage()

img_obj.load_file(data_dir=Dirs['images'], file_name=input_image)
img_obj.load_mask(mask_dir=Dirs['mask'], fget_mask=get_mask_file, erode=True)
img_obj.load_ground_truth(gt_dir=Dirs['truth'], fget_ground_truth=get_ground_truth_file)

img_obj.res['orig'] = img_obj.image_arr[:, :, 1]
img_obj.working_arr = img_obj.image_arr[:, :, 1]

img_obj.apply_mask()
img_obj.apply_bilateral(k_size=BILATERAL_KERNEL_SIZE, sig_color=BILATERAL_SIGMA_COLOR, sig_space=BILATERAL_SIGMA_SPACE)
img_obj.apply_gabor(filter_bank=kernels)

img_obj.generate_skeleton(threshold=params['sk_threshold'])
img_obj.generate_lattice_graph()
IMG.fromarray(img_obj.res['gabor'])

In [ ]:
img_obj.generate_skeleton(threshold=5)

In [ ]:
IMG.fromarray(img_obj.res['diff_bilateral'])

In [ ]:
tester = AtureTest(out_dir=Dirs['segmented'])
acc = tester.run(params=params, save_images=False, img_obj=img_obj)

In [ ]:
IMG.fromarray(imgutils.get_rgb_scores(img_obj.res['segmented'], img_obj.ground_truth))